In [1]:
import numpy as np
from scipy import linalg # had to update the scipy to 1.1 instead of 1.01 
# from sympy import Matrix
from scipy import sparse
import pandas as pd

In [2]:
# def read_sheet(pathtofile,sheetname):
#     data = pd.DataFrame(pathtofile,sheet_name =sheetname)

In [3]:
def find_h(reactor,timepoint,val_ls):
    print("Reactor: ",reactor)
    print("Time point: ", timepoint)
    
    # initialize list that will be used for the results of all the h's
    h_ls = [] # h, hco2, hh2, hbio, hch4
    
    # ------------------------------------------------
    # Assumptions
    # STEADY STATE
    # Two degrees of freedom... don't measure NH3 or H20 so make matrix to two rows
    error= 0.1 # assumed 10% measurement error
    X2 = 5.99 # chi-squared for 2 degrees of freedom at 95% confidence interval
    I = np.identity(2) # 2 degress of freedom for the identity matrix
    
    # Rows of "A" are C, H, O, N
    # Columns of "A" are CO2, H2, NH3, Biomass, H2O, CH4

    # stochiometric coefficients
    A = np.array([[-1,0,1,1], [2,-1,0.0265,2]])

    # ---------------------------------------------------------------------------------
    co2 = val_ls[0]
    h2 = val_ls[1]
    bio = val_ls[2]
    ch4 = val_ls[3]
       
    # measured variables
    x = np.array([[co2, h2, bio,ch4]]).T
#     print("A: ",A)
#     print("x: ",x)
    
    # vector of residuals
    E = -A.dot(x)
#     print("E: ",E)
    # variance-covariance matrix 
    
    # assume measurement error is independent and uncorrelated
    Y = np.array([[(co2*error)**2,0,0,0], [0,(h2*error)**2,0,0],[0,0,(bio*error)**2,0], [0,0,0,(ch4*error)**2]])
#     print("Y: ",Y)
    
    # net rate of input of components into system
    O = (A.dot(Y)).dot(A.T)
#     print("O: ",O)
    
    # inverse of net rate of input of components into system
    Oin = np.linalg.inv(O)
#     print("Oin: ", Oin)
    
    # test function
    h = (E.T.dot(Oin)).dot(E)
    hactual = h[0][0]
    h_ls.append(hactual)
#     print("h: ",hactual)
    
    # measurement error
    d = ((Y.dot(A.T)).dot(Oin)).dot(E)
#     print("d: ",d)
    
    # true values
    xt = x + d
#     print("xt: ",xt)

    # stop here if the hypothesis is passed ((chi_squared for 2 degrees of freedom) > htest)
    if hactual > X2:
        print("The test failed with an h value of: ", round(hactual,3))
        # and continue
    else:
        print("The test passed with an h value of: ", round(hactual,3))
    
    #--------------------------------------------------------------------------
    # Start finding the hs for the individual measurments
    
    # dictionary with which measurments to delete
    measure_list = ["co2","h2","bio","ch4"]
    
    # new product vectors without various conditions
    # CO2
    xuco2 = np.array([[h2, bio,ch4]]).T
    xcco2 = np.array([[co2]]).T

    # H2
    xuh2 = np.array([[co2, bio,ch4]]).T
    xch2 = np.array([[h2]]).T

    # Bio
    xubio = np.array([[co2, h2,ch4]]).T
    xcbio = np.array([[bio]]).T

    # CH4
    xuch4 = np.array([[co2, h2,bio]]).T
    xcch4 = np.array([[ch4]]).T

    measured_xu_dict = {"co2":xuco2,"h2":xuh2,"bio":xubio,"ch4":xuch4}
    measured_xc_dict = {"co2":xcco2,"h2":xch2,"bio":xcbio,"ch4":xcch4}
    
    # new variance-covariance matrices
    
    # CO2
    Yuco2 = np.array([[(h2*error)**2,0,0],[0,(bio*error)**2,0], [0,0,(ch4*error)**2]])
    Ycco2 = np.array([[(co2*error)**2]])

    # H2
    Yuh2 = np.array([[(co2*error)**2,0,0],[0,(bio*error)**2,0], [0,0,(ch4*error)**2]])
    Ych2 = np.array([[(h2*error)**2]])

    # Bio
    Yubio = np.array([[(co2*error)**2,0,0],[0,(h2*error)**2,0], [0,0,(ch4*error)**2]])
    Ycbio = np.array([[(bio*error)**2]])

    # CH4
    Yuch4 = np.array([[(co2*error)**2,0,0],[0,(h2*error)**2,0], [0,0,(bio*error)**2]])
    Ycch4 = np.array([[(ch4*error)**2]])

    measured_yu_dict = {"co2":Yuco2,"h2":Yuh2,"bio":Yubio,"ch4":Yuch4}
    measured_yc_dict = {"co2":Ycco2,"h2":Ych2,"bio":Ycbio,"ch4":Ycch4}
    
    #go through each gas
    for g in range(len(measure_list)):
        gas = measure_list[g]
        print(gas)
        xu = measured_xu_dict[gas]
        xc = measured_xc_dict[gas]
#         print(g)
#         print(xu)
#         print(xc)
        print("\n")

        Au = np.delete(A,g,axis=1)
        Ac = A[:,[g]]
#         print(Au)
#         print(Ac)
#         print("\n")

        Yu = measured_yu_dict[gas]
        Yc = measured_yc_dict[gas]
#         print(Yu)
#         print(Yc)
#         print("\n")

        scalar = (Ac.T.dot(Oin)).dot(Ac)
#         print(scalar)
        scalar_inv = np.linalg.inv(scalar)
#         print(scalar_inv)
#         print("\n")

        inbracketr = ((Ac.dot(scalar_inv)).dot(Ac.T)).dot(Oin)
#         print(inbracketr)
        Ouinv = Oin.dot(I-inbracketr)
        Ouinv = Ouinv.round(3)
#         print(Ouinv)
#         print("\n")
        ha = ((E.T).dot(Ouinv)).dot(E)
#         print(ha)
        hactual = ha[0][0]
        h_ls.append(hactual)
        if hactual > X2:
            print("The test failed with an h value of: ", round(hactual,3))
            # and continue
#         else:
#             print("The test passed with an h value of: ", round(hactual,3))
    return h_ls

In [4]:
def find_adjustments(reactor,timepoint,val_ls,drop_gas):
#     print("Reactor: ",reactor)
#     print("Time point: ", timepoint)
    
    # initialize list that will be used for the results of all the h's
#     h_ls = [] # h, hco2, hh2, hbio, hch4
    
    # ------------------------------------------------
    # Assumptions
    # STEADY STATE
    # Two degrees of freedom... don't measure NH3 or H20 so make matrix to two rows
    error= 0.1 # assumed 10% measurement error
    X2 = 5.99 # chi-squared for 2 degrees of freedom at 95% confidence interval
    I = np.identity(2) # 2 degress of freedom for the identity matrix
    
    # Rows of "A" are C, H, O, N
    # Columns of "A" are CO2, H2, NH3, Biomass, H2O, CH4

    # stochiometric coefficients
    A = np.array([[-1,0,1,1], [2,-1,0.0265,2]])

    # ---------------------------------------------------------------------------------
    co2 = val_ls[0]
    h2 = val_ls[1]
    bio = val_ls[2]
    ch4 = val_ls[3]
    
    # required to find adjustments
    val_ls_dict = {"co2":co2,"h2":h2,"bio":bio,"ch4":ch4}
       
    # measured variables
    x = np.array([[co2, h2, bio, ch4]]).T
#     print("A: ",A)
#     print("x: ",x)
    
    # vector of residuals
    E = -A.dot(x)
#     print("E: ",E)

#     # variance-covariance matrix 
#     # assume measurement error is independent and uncorrelated
    Y = np.array([[(co2*error)**2,0,0,0], [0,(h2*error)**2,0,0],[0,0,(bio*error)**2,0], [0,0,0,(ch4*error)**2]])
# #     print("Y: ",Y)
    
#     # net rate of input of components into system
    O = (A.dot(Y)).dot(A.T)
# #     print("O: ",O)
    
#     # inverse of net rate of input of components into system
    Oin = np.linalg.inv(O)
# #     print("Oin: ", Oin)
    
#     # test function
#     h = (E.T.dot(Oin)).dot(E)
#     hactual = h[0][0]
# #     h_ls.append(hactual)
# #     print("h: ",hactual)
    
#     # measurement error
#     d = ((Y.dot(A.T)).dot(Oin)).dot(E)
# #     print("d: ",d)
    
#     # true values
#     xt = x + d
#     print("xt: ",xt)

#     # stop here if the hypothesis is passed ((chi_squared for 2 degrees of freedom) > htest)
#     if hactual > X2:
#         print("The test failed with an h value of: ", round(hactual,3))
#         # and continue
#     else:
#         print("The test passed with an h value of: ", round(hactual,3))
    
    #--------------------------------------------------------------------------
    # Start finding the hs for the individual measurments
    
    # dictionary with which measurments to delete
#     measure_list = ["co2","h2","bio","ch4"]
    
    # new product vectors without various conditions
    # CO2
    xuco2 = np.array([[h2, bio,ch4]]).T
    xcco2 = np.array([[co2]]).T

    # H2
    xuh2 = np.array([[co2, bio,ch4]]).T
    xch2 = np.array([[h2]]).T

    # Bio
    xubio = np.array([[co2, h2,ch4]]).T
    xcbio = np.array([[bio]]).T

    # CH4
    xuch4 = np.array([[co2, h2,bio]]).T
    xcch4 = np.array([[ch4]]).T

    measured_xu_dict = {"co2":xuco2,"h2":xuh2,"bio":xubio,"ch4":xuch4}
    measured_xc_dict = {"co2":xcco2,"h2":xch2,"bio":xcbio,"ch4":xcch4}
    
    # new variance-covariance matrices
    
    # CO2
    Yuco2 = np.array([[(h2*error)**2,0,0],[0,(bio*error)**2,0], [0,0,(ch4*error)**2]])
    Ycco2 = np.array([[(co2*error)**2]])

    # H2
    Yuh2 = np.array([[(co2*error)**2,0,0],[0,(bio*error)**2,0], [0,0,(ch4*error)**2]])
    Ych2 = np.array([[(h2*error)**2]])

    # Bio
    Yubio = np.array([[(co2*error)**2,0,0],[0,(h2*error)**2,0], [0,0,(ch4*error)**2]])
    Ycbio = np.array([[(bio*error)**2]])

    # CH4
    Yuch4 = np.array([[(co2*error)**2,0,0],[0,(h2*error)**2,0], [0,0,(bio*error)**2]])
    Ycch4 = np.array([[(ch4*error)**2]])

    measured_yu_dict = {"co2":Yuco2,"h2":Yuh2,"bio":Yubio,"ch4":Yuch4}
    measured_yc_dict = {"co2":Ycco2,"h2":Ych2,"bio":Ycbio,"ch4":Ycch4}
    
#     #go through each gas
#     for g in range(len(measure_list)):
#         gas = measure_list[g]
#         print(gas)
    # defines which reactions to drop
    gas = drop_gas
    measure_dict = {"co2":0,"h2":1,"bio":2,"ch4":3}
    g = measure_dict[gas]
    xu = measured_xu_dict[gas]
    xc = measured_xc_dict[gas]
#         print(g)
#         print(xu)
#         print(xc)
#     print("\n")
    Au = np.delete(A,g,axis=1)
    Ac = A[:,[g]]
#         print(Au)
#         print(Ac)
#         print("\n")

    Yu = measured_yu_dict[gas]
    Yc = measured_yc_dict[gas]
#         print(Yu)
#         print(Yc)
#         print("\n")

    scalar = (Ac.T.dot(Oin)).dot(Ac)
#         print(scalar)
    scalar_inv = np.linalg.inv(scalar)
#         print(scalar_inv)
#         print("\n")

    inbracketr = ((Ac.dot(scalar_inv)).dot(Ac.T)).dot(Oin)
#         print(inbracketr)
    Ouinv = Oin.dot(I-inbracketr)
#     Ouinv = Ouinv.round(3)
#         print(Ouinv)
#         print("\n")
    ha = ((E.T).dot(Ouinv)).dot(E)
#         print(ha)
    hactual = ha[0][0]
    
    
    # measurement adjustments error
    du = ((Yu.dot(Au.T)).dot(Ouinv)).dot(E)
#     print("du",du)
    
    # convert to a list
    du_ls_temp = du.tolist()
#     du_ls_temp = pd.Series(du)
#     print(du_ls_temp)
    du_ls = []    
    for i in du_ls_temp:
#         print(i)
        i_new = list(i)
#         print(i_new[0])
        du_ls.append(i_new[0])
    du_dict = make_dict_gas(du_ls,gas)
        
    
#     h_ls.append(hactual)
#         if hactual > X2:
#             print("The test failed with an h value of: ", round(hactual,3))
#             # and continue
# #         else:
# #             print("The test passed with an h value of: ", round(hactual,3))
#     return h_ls
    return du_dict

In [5]:
def make_dict_gas(lst,dropped_gas):
#     print("Dropped gas: ", dropped_gas)
    gas_ls = ["co2","h2","bio","ch4"]
    gas_ls.remove(dropped_gas)
    dict_out ={}
    for g in range(len(gas_ls)):
        gas = gas_ls[g]
#         if gas != dropped_gas:
        dict_out[gas] = lst[g]
    return dict_out

In [6]:
def last_measurement(old_values_dict, adj_vector_dict, dropped_gas):

    if dropped_gas == "co2":
        print("CO2 dropped, calc new values")
        h2 = old_values_dict["h2"] + adj_vector_dict["h2"]
        bio = old_values_dict["bio"] + adj_vector_dict["bio"]
        ch4 = old_values_dict["ch4"] + adj_vector_dict["ch4"]
#         co2 = bio + ch4
        co2 = (h2-0.0265*bio - 2*ch4)/2
        result_ls = [co2,h2,bio,ch4]
        
    elif dropped_gas == "h2":
        print("H2 dropped, calc new values")
        co2 = old_values_dict["co2"] + adj_vector_dict["co2"]
        bio = old_values_dict["bio"] + adj_vector_dict["bio"]
        ch4 = old_values_dict["ch4"] + adj_vector_dict["ch4"]
        h2 = 2*co2 + 0.0265*bio + 2*ch4
        result_ls = [co2,h2,bio,ch4]
    elif dropped_gas == "bio":
        print("Bio dropped, calc new values")
        co2 = old_values_dict["co2"] + adj_vector_dict["co2"]
        h2 = old_values_dict["h2"] + adj_vector_dict["h2"]
        ch4 = old_values_dict["ch4"] + adj_vector_dict["ch4"]
#         bio = co2 - ch4
        bio = (h2-2*co2 - 2*ch4)/0.0265

        result_ls = [co2,h2,bio,ch4]
    elif dropped_gas == "ch4":
        print("CH4 dropped, calc new values")
        co2 = old_values_dict["co2"] + adj_vector_dict["co2"]
        h2 = old_values_dict["h2"] + adj_vector_dict["h2"]
        bio = old_values_dict["bio"] + adj_vector_dict["bio"]
#         ch4 = co2 - bio
        ch4 = (h2-2*co2 - 0.0265*bio)/2

        result_ls = [co2,h2,bio,ch4]
    else:
        print("error in dropped value")
        result_ls = []
    return result_ls        

## Trial with M.t.

### biomass - CO2 - H2 + CH4 + H2O - NH3 = 0
#### ignore Sulfur in biomass
### CH<sub>1.681</sub>O<sub>0.481</sub>N<sub>0.222</sub>S<sub>0.0004</sub> -YCO<sub>2</sub> - YH<sub>2</sub> + YCH<sub>4</sub> + YH<sub>2</sub>O - YNH<sub>3</sub> = 0
#### unknowns are H2O and NH3

# Example from paper Wang 1983

In [12]:
# Read in Data
# should have this format each column is a reactor
# the rows should be the CO2, H2, Biomass, CH4
# Units should all be the same (mmol/h)

pathtodata = r"F:\Experimental_Work\Reactors_Ley\F6_Exp2\Error_Analysis_Data_SS2.xlsx"
pathtoresult = r"F:\Experimental_Work\Reactors_Ley\F6_Exp2\Error_Analysis_h_Results_SS2.xlsx"
pathtoresult_drop = r"F:\Experimental_Work\Reactors_Ley\F6_Exp2\Error_Analysis_h_Results_drop_SS2.xlsx"

In [13]:
df = pd.read_excel(pathtodata, sheet_name = None,engine="openpyxl",index_col="Timepoint") # read all sheets

In [14]:
sheet_ls = ["S1R2","S2R1","S2R3","S2R7","S1R4","S1R5","S2R6","S2R8","S1R7","S2R2","S2R4","S2R5"]
time_point_ls = [4.1,4.8,5.6,6.8]

In [15]:
h_dict= {}
for i in sheet_ls:
    print(i)
    df_temp = df[i]
#     print(df_temp)
    temp_dict = {}
    for t in time_point_ls:
        col = df_temp[t]
#         print(col)
        co2 = df_temp[t].loc["CO2"]
        h2 = df_temp[t].loc["H2"]
        bio = df_temp[t].loc["Biomass"]
        ch4 = df_temp[t].loc["CH4"]
        val_ls = [co2,h2,bio,ch4]
#         print("CO2: ",co2)
#         print("H2: ",h2)
#         print("Biomass: ",bio)
#         print("CH4: ",ch4)
        h_list = find_h(reactor=i,timepoint=t,val_ls=val_ls)
        print(h_list)
        temp_dict[t]=h_list
    h_dict[i]=temp_dict
print("h_dict:\n",h_dict)

S1R2
Reactor:  S1R2
Time point:  4.1
The test passed with an h value of:  3.453
co2


h2


bio


ch4


[3.4525059418758532, 2.1638826989789406, 2.5515173310010746, 0.4533841991576875, 0.024858611864535834]
Reactor:  S1R2
Time point:  4.8
The test passed with an h value of:  4.078
co2


h2


bio


ch4


[4.078448070483947, 4.068820972290023, 0.7056388338346563, 3.0260525402814, 1.3220234870860157]
Reactor:  S1R2
Time point:  5.6
The test passed with an h value of:  5.011
co2


h2


bio


ch4


[5.011363251979268, 4.5496241586926445, 1.9240097112714403, 2.395286055654171, 0.5297511277508339]
Reactor:  S1R2
Time point:  6.8
The test passed with an h value of:  3.941
co2


h2


bio


ch4


[3.9409663866578724, 2.0756816942710508, 3.2161954267513413, 0.25526257484944936, 0.15370170608089917]
S2R1
Reactor:  S2R1
Time point:  4.1
The test passed with an h value of:  1.895
co2


h2


bio


ch4


[1.894828354096774, 1.8775588794875984, 0.45701408753933587, 1.2864623144899556, 0.4534102858153997

In [16]:
# df_ls = []
with pd.ExcelWriter(pathtoresult) as writer:
    for r in sheet_ls:
        data = h_dict[r]
    #     print(data)
        df_temp = pd.DataFrame.from_dict(h_dict[r])
        df_temp = df_temp.set_index([pd.Index(["h", "h_co2","h_h2","h_bio","h_ch4"])])
        print(df_temp)
        
        df_temp.to_excel(writer,sheet_name=r)  

            4.1       4.8       5.6       6.8
h      3.452506  4.078448  5.011363  3.940966
h_co2  2.163883  4.068821  4.549624  2.075682
h_h2   2.551517  0.705639  1.924010  3.216195
h_bio  0.453384  3.026053  2.395286  0.255263
h_ch4  0.024859  1.322023  0.529751  0.153702
            4.1       4.8       5.6       6.8
h      1.894828  1.219360  2.546981  3.097722
h_co2  1.877559  1.097288  2.155340  1.846478
h_h2   0.457014  0.018987  1.306755  2.392789
h_bio  1.286462  1.180678  0.923661  0.337670
h_ch4  0.453410  0.778269  0.092002  0.046602
            4.1       4.8       5.6       6.8
h      0.610385  1.118452  2.730608  0.769730
h_co2  0.579494  1.118333  2.731083  0.682360
h_h2   0.238278  0.199661  0.420651  0.382614
h_bio  0.323997  0.847522  2.116347  0.317155
h_ch4  0.066764  0.345294  0.950500  0.036707
            4.1       4.8       5.6       6.8
h      0.028803  1.061193  0.095712  2.441912
h_co2  0.022726  0.009220  0.087052  0.513924
h_h2   0.000029  0.861251  0.00316

In [17]:
index_val_list = ["all","co2","h2","bio","ch4"]
h_dict= {}
for i in sheet_ls:
    print(i)
    df_temp = df[i]
#     print(df_temp)
    temp_dict = {}
    for t in time_point_ls:
        col = df_temp[t]
#         print(col)
        co2 = df_temp[t].loc["CO2"]
        h2 = df_temp[t].loc["H2"]
        bio = df_temp[t].loc["Biomass"]
        ch4 = df_temp[t].loc["CH4"]
        val_ls = [co2,h2,bio,ch4]
#         print("CO2: ",co2)
#         print("H2: ",h2)
#         print("Biomass: ",bio)
#         print("CH4: ",ch4)
        h_list = find_h(reactor=i,timepoint=t,val_ls=val_ls)
        print(h_list)
        temp_dict[t]=h_list
        if h_list[0] < 5.99: #95% confidence interval with 2 degrees of freedom
            pass
        else:
            min_val = min(h_list)
            index = h_list.index(min_val)
            print("Min_val",min_val)
            print(index)
            print("Drop this measurement: ",index_val_list[index])
            h_list[index] = "DROP"
            
    h_dict[i]=temp_dict
print("h_dict:\n",h_dict)

S1R2
Reactor:  S1R2
Time point:  4.1
The test passed with an h value of:  3.453
co2


h2


bio


ch4


[3.4525059418758532, 2.1638826989789406, 2.5515173310010746, 0.4533841991576875, 0.024858611864535834]
Reactor:  S1R2
Time point:  4.8
The test passed with an h value of:  4.078
co2


h2


bio


ch4


[4.078448070483947, 4.068820972290023, 0.7056388338346563, 3.0260525402814, 1.3220234870860157]
Reactor:  S1R2
Time point:  5.6
The test passed with an h value of:  5.011
co2


h2


bio


ch4


[5.011363251979268, 4.5496241586926445, 1.9240097112714403, 2.395286055654171, 0.5297511277508339]
Reactor:  S1R2
Time point:  6.8
The test passed with an h value of:  3.941
co2


h2


bio


ch4


[3.9409663866578724, 2.0756816942710508, 3.2161954267513413, 0.25526257484944936, 0.15370170608089917]
S2R1
Reactor:  S2R1
Time point:  4.1
The test passed with an h value of:  1.895
co2


h2


bio


ch4


[1.894828354096774, 1.8775588794875984, 0.45701408753933587, 1.2864623144899556, 0.4534102858153997

In [18]:
# df_ls = []
with pd.ExcelWriter(pathtoresult_drop) as writer:
    for r in sheet_ls:
        data = h_dict[r]
    #     print(data)
        df_temp = pd.DataFrame.from_dict(h_dict[r])
        df_temp = df_temp.set_index([pd.Index(["h", "h_co2","h_h2","h_bio","h_ch4"])])
        print(df_temp)
        
        df_temp.to_excel(writer,sheet_name=r)  

            4.1       4.8       5.6       6.8
h      3.452506  4.078448  5.011363  3.940966
h_co2  2.163883  4.068821  4.549624  2.075682
h_h2   2.551517  0.705639  1.924010  3.216195
h_bio  0.453384  3.026053  2.395286  0.255263
h_ch4  0.024859  1.322023  0.529751  0.153702
            4.1       4.8       5.6       6.8
h      1.894828  1.219360  2.546981  3.097722
h_co2  1.877559  1.097288  2.155340  1.846478
h_h2   0.457014  0.018987  1.306755  2.392789
h_bio  1.286462  1.180678  0.923661  0.337670
h_ch4  0.453410  0.778269  0.092002  0.046602
            4.1       4.8       5.6       6.8
h      0.610385  1.118452  2.730608  0.769730
h_co2  0.579494  1.118333  2.731083  0.682360
h_h2   0.238278  0.199661  0.420651  0.382614
h_bio  0.323997  0.847522  2.116347  0.317155
h_ch4  0.066764  0.345294  0.950500  0.036707
            4.1       4.8       5.6       6.8
h      0.028803  1.061193  0.095712  2.441912
h_co2  0.022726  0.009220  0.087052  0.513924
h_h2   0.000029  0.861251  0.00316

# adjust the values by dropping the measurment with the lowest h value

In [19]:
df

{'S1R2':                 4.1       4.8       5.6       6.8
 Timepoint                                        
 CO2        1.240481  1.017427  1.134527  1.177639
 H2         4.714991  4.633977  4.887702  4.319393
 Biomass    0.070511  0.073446  0.068890  0.071214
 CH4        0.922754  0.833334  0.867319  0.846062,
 'S2R1':                 4.1       4.8       5.6       6.8
 Timepoint                                        
 CO2        1.527492  1.367042  1.630508  1.718637
 H2         6.539482  6.006027  6.621595  6.487697
 Biomass    0.087782  0.090639  0.092394  0.092492
 CH4        1.303815  1.250840  1.299531  1.293135,
 'S2R3':                 4.1       4.8       5.6       6.8
 Timepoint                                        
 CO2        1.411692  1.403002  1.391838  1.483083
 H2         5.690302  5.906588  6.189114  5.914516
 Biomass    0.082978  0.093327  0.096333  0.090458
 CH4        1.237001  1.226308  1.177113  1.271653,
 'S2R7':                 4.1       4.8       5.6       

In [20]:
# read in the results from the error analysis
df_h = pd.read_excel(pathtoresult, sheet_name = None,engine="openpyxl",index_col=0) # read all sheets

In [21]:
df_h

{'S1R2':             4.1       4.8       5.6       6.8
 h      3.452506  4.078448  5.011363  3.940966
 h_co2  2.163883  4.068821  4.549624  2.075682
 h_h2   2.551517  0.705639  1.924010  3.216195
 h_bio  0.453384  3.026053  2.395286  0.255263
 h_ch4  0.024859  1.322023  0.529751  0.153702,
 'S2R1':             4.1       4.8       5.6       6.8
 h      1.894828  1.219360  2.546981  3.097722
 h_co2  1.877559  1.097288  2.155340  1.846478
 h_h2   0.457014  0.018987  1.306755  2.392789
 h_bio  1.286462  1.180678  0.923661  0.337670
 h_ch4  0.453410  0.778269  0.092002  0.046602,
 'S2R3':             4.1       4.8       5.6       6.8
 h      0.610385  1.118452  2.730608  0.769730
 h_co2  0.579494  1.118333  2.731083  0.682360
 h_h2   0.238278  0.199661  0.420651  0.382614
 h_bio  0.323997  0.847522  2.116347  0.317155
 h_ch4  0.066764  0.345294  0.950500  0.036707,
 'S2R7':             4.1       4.8       5.6       6.8
 h      0.028803  1.061193  0.095712  2.441912
 h_co2  0.022726  0.00922

In [ ]:
# h_gas_dict = {"h_co2":"co2","h_h2":"h2","h_bio":"bio","h_ch4":"ch4"}
# for i in sheet_ls:
#     print(i)
#     # for each sheet in the gas data
#     df_temp = df[i]
#     df_h_temp = df_h[i]
#     for t in time_point_ls:
#         col = df_h_temp[t]
#         col_dict = col.to_dict()
#         col_dict_flip = {value:key for key, value in col_dict.items()}
#         print(col_dict_flip)
# #         print(type(col))
# #         col_temp = col[1:]
#         min_h = min(col[1:])# only search after the h value = h_co2-->h_ch4
# #         print(min_h)
#         h_to_drop = col_dict_flip[min_h]
# #         print(h_to_drop)
#         print("measurement to drop: ", h_gas_dict[h_to_drop])
    
    
    

    

In [22]:
# h_dict= {}
df_out = df # make the new dataframe with the updated measurment values
h_gas_dict = {"h_co2":"co2","h_h2":"h2","h_bio":"bio","h_ch4":"ch4"}
for i in sheet_ls:
    print("Reactor: ",i)
    df_temp = df[i] # raw data
    df_h_temp = df_h[i] # data with the h values
    df_out_temp = df_out[i]
    print("df_out_temp_old",df_out_temp)

#     print(df_temp)
#     temp_dict = {}
    for t in time_point_ls:
        print("Time Point: ",t)
        col = df_temp[t] # data for the time point
#         print(col)
        co2 = df_temp[t].loc["CO2"]
        h2 = df_temp[t].loc["H2"]
        bio = df_temp[t].loc["Biomass"]
        ch4 = df_temp[t].loc["CH4"]
        val_ls = [co2,h2,bio,ch4]
        val_dict = {"co2":co2,"h2":h2,"bio":bio,"ch4":ch4}
        print("Old Value Dictionary", val_dict)
#         print("CO2: ",co2)
#         print("H2: ",h2)
#         print("Biomass: ",bio)
#         print("CH4: ",ch4)
        col_h = df_h_temp[t]
        col_dict = col_h.to_dict()
        col_dict_flip = {value:key for key, value in col_dict.items()}
        print(col_dict_flip)
#         print(type(col))
#         col_temp = col[1:]
        min_h = min(col_h[1:])# only search after the h value = h_co2-->h_ch4
#         print(min_h)
        h_to_drop = col_dict_flip[min_h]
#         print(h_to_drop)
        print("measurement to drop: ", h_gas_dict[h_to_drop])


        adj_vector_dict = find_adjustments(reactor=i,timepoint=t,val_ls=val_ls,drop_gas=h_gas_dict[h_to_drop])
        print(adj_vector_dict)
        updated_gas_measurement_ls = last_measurement(old_values_dict=val_dict,adj_vector_dict=adj_vector_dict,dropped_gas=h_gas_dict[h_to_drop])
        print("Updated_gas_measurements",updated_gas_measurement_ls)
        df_out_temp[t]=updated_gas_measurement_ls
        print("df_out_temp updated: ",df_out_temp)
        print("\n")
        
#         print(h_list)
#         temp_dict[t]=h_list
#     h_dict[i]=temp_dict
# print("h_dict:\n",h_dict)

Reactor:  S1R2
df_out_temp_old                 4.1       4.8       5.6       6.8
Timepoint                                        
CO2        1.240481  1.017427  1.134527  1.177639
H2         4.714991  4.633977  4.887702  4.319393
Biomass    0.070511  0.073446  0.068890  0.071214
CH4        0.922754  0.833334  0.867319  0.846062
Time Point:  4.1
Old Value Dictionary {'co2': 1.2404808817079265, 'h2': 4.714990813429642, 'bio': 0.07051062377858545, 'ch4': 0.9227544586429157}
{3.452505941875853: 'h', 2.163882698978941: 'h_co2', 2.551517331001075: 'h_h2', 0.4533841991576875: 'h_bio', 0.02485861186453583: 'h_ch4'}
measurement to drop:  ch4
{'co2': -0.014153777338842384, 'h2': 0.051120361823816296, 'bio': 2.2562045016346694e-05}
CH4 dropped, calc new values
Updated_gas_measurements [1.226327104369084, 4.766111175253458, 0.0705331858236018, 1.1557939185454824]
df_out_temp updated:                  4.1       4.8       5.6       6.8
Timepoint                                        
CO2        1.

In [23]:
pathtoresult_edited = r"F:\Experimental_Work\Reactors_Ley\F6_Exp2\Error_Analysis_Data_adjusted_SS2.xlsx"


In [24]:
# df_ls = []
with pd.ExcelWriter(pathtoresult_edited) as writer:
    for r in sheet_ls:
        df_temp = df_out[r]
#         data = h_dict[r]
#     #     print(data)
#         df_temp = pd.DataFrame.from_dict(h_dict[r])
#         df_temp = df_temp.set_index([pd.Index(["h", "h_co2","h_h2","h_bio","h_ch4"])])
#         print(df_temp)
        
        df_temp.to_excel(writer,sheet_name=r,engine="openpyxl")